In [ ]:
# Instruction: 从头运行到loop work cell，之后修改日期，反复运行从loop work cell至最终的cells

In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

if(not sc and not spark):
    sc = SparkContext('local')
    spark = SparkSession(sc)

In [21]:
import fileinput

In [30]:
uid_iid_path = "./data/uid_iid"
train_set_path = "./data/tianchi_fresh_comp_train_user.csv"
feature_path = "./data/feature/"

In [2]:
"""
data = sc.textFile(train_set_path, 1)
"""

In [ ]:
####################################################################################
############################         数据预处理           ############################
####################################################################################

In [3]:
# data.count()

23291028

In [4]:
# data.top(10)

[u'user_id,item_id,behavior_type,user_geohash,item_category,time',
 u'65361888,98907464,1,,9559,2014-12-13 04',
 u'65361888,98907464,1,,9559,2014-12-13 04',
 u'65361888,9679068,1,,4116,2014-12-04 03',
 u'65361888,96612624,1,,2993,2014-12-10 05',
 u'65361888,96612624,1,,2993,2014-12-10 05',
 u'65361888,96612624,1,,2993,2014-12-10 05',
 u'65361888,96584710,2,,11824,2014-12-14 07',
 u'65361888,96584710,1,,11824,2014-12-14 07',
 u'65361888,92945029,1,,434,2014-12-08 14']

In [5]:
"""
dataheader = data.filter(lambda l: 'user_id' in l)
"""

In [6]:
"""
dataheader.collect()
"""

[u'user_id,item_id,behavior_type,user_geohash,item_category,time']

In [7]:
"""
datanoheader = data.subtract(dataheader)
"""

In [8]:
# datanoheader.count()

23291027

In [9]:
"""
def extract(line):
    import time
    try:
        part = line.strip().split(",")
        uid, iid, beh, ict, time = part[0], part[1], part[2], part[4], "-".join(part[5].split(" ")[0].split("-")[1:])+"-"+part[5].split(" ")[1]
        return ((uid, iid, ict), time+","+beh)
    except:
        return ((""), "")
"""

In [10]:
"""
counts = datanoheader.map(lambda x : extract(x)) \
                .filter(lambda x : x[0]!="") \
                .groupByKey() \
                .map(lambda x : (" ".join(x[0])+"\t"+" ".join([str(item["time"])+","+item["beh"] for item in sorted([{"time":content.split(",")[0],"beh":content.split(",")[1]} for content in x[1]],key=lambda x:x["time"])])))
"""

In [14]:
"""
output = counts.saveAsTextFile(uid_iid_path)
"""

In [15]:
####################################################################################
############################           商品特征           ############################
####################################################################################

In [16]:
def extract1_item(line):
    import time
    (uid, iid, ict) = line.strip().split("\t")[0].split(" ")
    if subset.has_key(iid):
        items = filter(lambda x:x[0]>0, [(int(time.mktime(time.strptime('2014-'+etime,'%Y-%m-%d-%H'))-time.mktime(time.strptime('2014-'+i.split(",")[0],'%Y-%m-%d-%H')))/(24*3600)+1, int(i.split(",")[1])) for i in line.strip().split("\t")[1].split(" ")])
        return (iid,items)
    else:
        return ("","")

In [17]:
def extract2_item(items_list):
    items, items_buy, items_buy_3, f, inf = [], [], [], [0]*37, 100
    f[30] = len(items_list) # 交互人数
    for i in items_list:
        if len(filter(lambda x:x[1]==4,i))>0:
            items_buy.append(i)
        if len(filter(lambda x:x[1]==4 and x[0]<=3,i))>0:
            items_buy_3.append(i)
        items.extend(i)
    f[31] = len(items_buy) # 购买人数
    f[32] = len(items_buy_3) # 三天内购买人数
    f[33] = len(filter(lambda x:len(x)==1,items_list)) # 只有过一次交互的用户数
    f[34] = len(filter(lambda x:len(x)==2,items_list)) # 有过两次交互的用户数
    f[35] = len(filter(lambda x:len(x)==3,items_list)) # 有过三次交互的用户数
    items = sorted(items, key=lambda x:x[0], reverse=True)
    f[0] = len(filter(lambda x:x[0]<=1 and x[1]==1, items)) # 最后1天点击次数
    f[1] = len(filter(lambda x:x[0]<=1 and x[1]==2, items)) # 最后1天加收次数
    f[2] = len(filter(lambda x:x[0]<=1 and x[1]==3, items)) # 最后1天加购次数
    f[3] = len(filter(lambda x:x[0]<=1 and x[1]==4, items)) # 最后1天购买次数
    f[4] = len(filter(lambda x:x[0]==2 and x[1]==1, items)) # 倒数第2天点击次数
    f[5] = len(filter(lambda x:x[0]==2 and x[1]==2, items)) # 倒数第2天加收次数
    f[6] = len(filter(lambda x:x[0]==2 and x[1]==3, items)) # 倒数第2天加购次数
    f[7] = len(filter(lambda x:x[0]==2 and x[1]==4, items)) # 倒数第2天购买次数
    f[8] = len(filter(lambda x:x[0]==3 and x[1]==1, items)) # 倒数第3天点击次数
    f[9] = len(filter(lambda x:x[0]==3 and x[1]==2, items)) # 倒数第3天加收次数
    f[10] = len(filter(lambda x:x[0]==3 and x[1]==3, items)) # 倒数第3天加购次数
    f[11] = len(filter(lambda x:x[0]==3 and x[1]==4, items)) # 倒数第3天购买次数
    f[12] = len(filter(lambda x:x[0]<=7 and x[1]==1, items)) # 最后1周点击次数
    f[13] = len(filter(lambda x:x[0]<=7 and x[1]==2, items)) # 最后1周加收次数
    f[14] = len(filter(lambda x:x[0]<=7 and x[1]==3, items)) # 最后1周加购次数
    f[15] = len(filter(lambda x:x[0]<=7 and x[1]==4, items)) # 最后1周购买次数
    f[16] = len(filter(lambda x:x[0]<=14 and x[1]==1, items)) # 最后2周点击次数
    f[17] = len(filter(lambda x:x[0]<=14 and x[1]==2, items)) # 最后2周加收次数
    f[18] = len(filter(lambda x:x[0]<=14 and x[1]==3, items)) # 最后2周加购次数
    f[19] = len(filter(lambda x:x[0]<=14 and x[1]==4, items)) # 最后2周购买次数
    f[20] = min(1.0,round(1.0*(f[3]+f[7]+f[11])/(f[0]+f[4]+f[8]),4)) if (f[0]+f[4]+f[8]) else 0.0 # 最后3天点击转化率
    f[21] = min(1.0,round(1.0*(f[3]+f[7]+f[11])/(f[1]+f[5]+f[9]),4)) if (f[1]+f[5]+f[9])!=0 else 0.0 # 最后3天加收转化率
    f[22] = min(1.0,round(1.0*(f[3]+f[7]+f[11])/(f[2]+f[6]+f[10]),4)) if f[2]!=0 else 0.0 # 最后3天加购转化率
    f[23] = min(1.0,round(1.0*f[7]/f[4],4)) if f[4]!=0 else 0.0 # 最后2周点击转化率
    f[24] = min(1.0,round(1.0*f[7]/f[5],4)) if f[5]!=0 else 0.0 # 最后2周加收转化率
    f[25] = min(1.0,round(1.0*f[7]/f[6],4)) if f[6]!=0 else 0.0 # 最后2周加购转化率
    buy = filter(lambda x:x[1]==4, items)
    last = buy[-1][0] if len(buy)!=0 else inf
    f[26] = len(buy) # 购买总次数 
    f[27] = len(filter(lambda x:x[0]==last and x[1]==1, items)) # 最后一次发生购买的当天发生的点击次数
    f[28] = len(filter(lambda x:x[0]==last and x[1]==2, items)) # 最后一次发生购买的当天发生的加收次数
    f[29] = len(filter(lambda x:x[0]==last and x[1]==3, items)) # 最后一次发生购买的当天发生的加购次数
    f[36] = round(1.0*len(items)/f[30],4) if f[30]!=0 else 0.0 # 人均交互次数
    return "\t".join([str(i) for i in f])

In [18]:
global etime
global subset

In [19]:
lines = sc.textFile(uid_iid_path, 1)

In [25]:
####################################################################################
############################           用户特征           ###########################
####################################################################################

In [26]:
def extract1_user(line):
    import time
    (uid, iid, ict) = line.strip().split("\t")[0].split(" ")
    items = filter(lambda x:x[0]>0, [(int(time.mktime(time.strptime('2014-'+etime,'%Y-%m-%d-%H'))-time.mktime(time.strptime('2014-'+i.split(",")[0],'%Y-%m-%d-%H')))/(24*3600)+1, int(i.split(",")[1])) for i in line.strip().split("\t")[1].split(" ")])
    return (uid,items)

In [27]:
def extract2_user(items_list):
    import itertools
    items, items_buy, items_buy_3, f, inf = [], [], [], [0]*39, 100
    f[32] = len(items_list) # 交互商品数
    for i in items_list:
        if len(filter(lambda x:x[1]==4,i))>0:
            items_buy.append(i)
        if len(filter(lambda x:x[1]==4 and x[0]<=3,i))>0:
            items_buy_3.append(i)	
        items.extend(i)
    f[33] = len(items_buy) # 购买商品数
    f[34] = len(items_buy_3) # 三天内购买商品数
    f[35] = len(filter(lambda x:len(x)==1,items_list)) # 只有过一次交互的商品数
    f[36] = len(filter(lambda x:len(x)==2,items_list)) # 有过两次交互的商品数
    f[37] = len(filter(lambda x:len(x)==3,items_list)) # 有过三次交互的商品数
    items = sorted(items, key=lambda x:x[0], reverse=True)
    buy = filter(lambda x:x[1]==4, items)
    last = buy[-1][0] if len(buy)!=0 else inf
    f[24] = len(filter(lambda x:x[0]<=1 and x[1]==1, items)) # 最后1天点击次数
    f[25] = len(filter(lambda x:x[0]<=1 and x[1]==2, items)) # 最后1天加收次数
    f[26] = len(filter(lambda x:x[0]<=1 and x[1]==3, items)) # 最后1天加购次数
    f[27] = len(filter(lambda x:x[0]<=1 and x[1]==4, items)) # 最后1天购买次数
    f[28] = len(filter(lambda x:x[0]<=3 and x[1]==1, items)) # 最后3天点击次数
    f[29] = len(filter(lambda x:x[0]<=3 and x[1]==2, items)) # 最后3天加收次数
    f[30] = len(filter(lambda x:x[0]<=3 and x[1]==3, items)) # 最后3天加购次数
    f[31] = len(filter(lambda x:x[0]<=3 and x[1]==4, items)) # 最后3天购买次数
    f[0] = len(filter(lambda x:x[0]<=7 and x[1]==1, items)) # 最后1周点击次数
    f[1] = len(filter(lambda x:x[0]<=7 and x[1]==2, items)) # 最后1周加收次数
    f[2] = len(filter(lambda x:x[0]<=7 and x[1]==3, items)) # 最后1周加购次数
    f[3] = len(filter(lambda x:x[0]<=7 and x[1]==4, items)) # 最后1周购买次数
    f[4] = len(filter(lambda x:x[0]<=21 and x[1]==1, items)) # 最后3周点击次数
    f[5] = len(filter(lambda x:x[0]<=21 and x[1]==2, items)) # 最后3周加收次数
    f[6] = len(filter(lambda x:x[0]<=21 and x[1]==3, items)) # 最后3周加购次数
    f[7] = len(filter(lambda x:x[0]<=21 and x[1]==4, items)) # 最后3周购买次数
    f[8] = min(1.0,round(1.0*f[3]/f[0],4)) if f[0]!=0 else 0.0 # 最后1周点击转化率
    f[9] = min(1.0,round(1.0*f[3]/f[1],4)) if f[1]!=0 else 0.0 # 最后1周加收转化率
    f[10] = min(1.0,round(1.0*f[3]/f[2],4)) if f[2]!=0 else 0.0 # 最后1周加购转化率
    f[11] = min(1.0,round(1.0*f[7]/f[4],4)) if f[4]!=0 else 0.0 # 最后3周点击转化率
    f[12] = min(1.0,round(1.0*f[7]/f[5],4)) if f[5]!=0 else 0.0 # 最后3周加收转化率
    f[13] = min(1.0,round(1.0*f[7]/f[6],4)) if f[6]!=0 else 0.0 # 最后3周加购转化率
    f[14] = last # 最后一次购买距离天数
    f[15] = len(set([item[0] for item in items if item[0]<=3])) # 最后3天内交互天数
    f[16] = len(set([item[0] for item in items if item[0]<=7])) # 最后1周内交互天数
    f[17] = len(set([item[0] for item in items if item[0]<=21])) # 最后3周内交互天数
    f[18] = items[-1][0] if len(items)!=0 else inf # 最后1次交互距离天数
    inter = [len(list(i)) for _,i in itertools.groupby(items, lambda x: x[0])]
    f[19] = len(inter) #交互天数
    f[20] = max(inter) if len(inter)!=0 else 0 #交互最多的一天交互次数
    f[21] = len(filter(lambda x:x[0]<=1 and x[1]==4, items)) # 最后1天购买次数
    f[22] = len(filter(lambda x:x[0]<=3 and x[1]==4, items)) # 最后3天购买次数
    f[23] = len(filter(lambda x:x[0]<=7 and x[1]==4, items)) # 最后7天购买次数
    f[38] = round(1.0*len(items)/f[32],4) if f[32]!=0 else 0.0 # 用户对每件商品的平均交互次数
    return "\t".join([str(i) for i in f])

In [ ]:
# lines = sc.textFile(uid_iid_path, 1)

In [34]:
####################################################################################
############################       用户-品牌特征          ###########################
####################################################################################

In [35]:
def extract1_user_brand(line):
    import time
    (uid, iid, ict) = line.strip().split("\t")[0].split(" ")
    if subset.has_key(ict):
        items = filter(lambda x:x[0]>0, [(int(time.mktime(time.strptime('2014-'+etime,'%Y-%m-%d-%H'))-time.mktime(time.strptime('2014-'+i.split(",")[0],'%Y-%m-%d-%H')))/(24*3600)+1, int(i.split(",")[1])) for i in line.strip().split("\t")[1].split(" ")])
        return (uid+"\t"+ict,items)
    else:
        return ("","")

In [36]:
def extract2_user_brand(items_list):
    import itertools
    items = []
    for i in items_list:
        items.extend(i)
    items = sorted(items, key=lambda x:x[0], reverse=True) if len(items)!=0 else []
    f, inf = [0]*26, 100
    buy = filter(lambda x:x[1]==4, items)
    last = buy[-1][0] if len(buy)!=0 else inf
    a1 = filter(lambda x:x[0]<last and x[1]==1, items)
    a2 = filter(lambda x:x[0]<last and x[1]==2, items)
    a3 = filter(lambda x:x[0]<last and x[1]==3, items)
    # 基本统计特征
    f[0] = a2[-1][0] if len(a2)!=0 else inf # 购买后最后一次加入收藏夹距离天数
    f[1] = a3[-1][0] if len(a3)!=0 else inf # 购买后最后一次加入购物车距离天数
    f[2] = len(a1) # 购买后点击次数
    f[3] = len(a2) # 购买后加收次数
    f[4] = len(a3) # 购买后加购次数
    f[5] = len(filter(lambda x:x[0]<=1, items)) # 最后1天交互次数
    f[6] = len(filter(lambda x:x[0]==2, items)) # 倒数第2天交互次数
    f[7] = len(filter(lambda x:x[0]==3, items)) # 倒数第3天交互次数
    f[8] = len(filter(lambda x:x[0]==4, items)) # 倒数第4天交互次数
    f[9] = len(filter(lambda x:x[0]<=7, items)) # 最后7天交互次数
    f[10] = len(buy) # 历史购买次数
    f[11] = last # 最后一次购买距离天数
    f[12] = len(set([item[0] for item in items if item[0]<=3])) # 最后3天内交互天数
    f[13] = len(set([item[0] for item in items if item[0]<=7])) # 最后1周内交互天数
    f[14] = len(set([item[0] for item in items if item[0]<=21])) # 最后3周内交互天数
    f[15] = items[-1][0] if len(items)!=0 else inf # 最后1次交互距离天数
    inter = [len(list(i)) for _,i in itertools.groupby(items, lambda x: x[0])]
    f[16] = len(inter) #交互天数
    f[17] = max(inter) if len(inter)!=0 else 0 #交互最多的一天交互次数
    f[18] = len(filter(lambda x:x[0]<=1 and x[1]==4, items)) # 最后1天购买次数
    f[19] = len(filter(lambda x:x[0]<=2 and x[1]==4, items)) # 最后2天购买次数
    f[20] = len(filter(lambda x:x[0]<=3 and x[1]==4, items)) # 最后3天购买次数
    f[21] = len(filter(lambda x:x[0]<=7 and x[1]==4, items)) # 最后7天购买次数
    f[22] = len(filter(lambda x:x[1]==1, items)) # 总点击次数
    f[23] = len(filter(lambda x:x[1]==2, items)) # 总添加收藏夹次数
    f[24] = len(filter(lambda x:x[1]==3, items)) # 总加购次数
    f[25] = items[0][0]-items[-1][0]+1 if len(items)!=0 else 0 # 第一次交互到最后一次交互的持续时间
    return "\t".join([str(i) for i in f])

In [ ]:
# lines = sc.textFile(uid_iid_path, 1)

In [ ]:
####################################################################################
############################       用户-商品特征          ###########################
####################################################################################

In [40]:
def extract_user_prod(line):
    import time
    import itertools
    (uid, iid, ict) = line.strip().split("\t")[0].split(" ")
    if subset.has_key(iid):
        items = filter(lambda x:x[0]>0, [(int(time.mktime(time.strptime('2014-'+etime,'%Y-%m-%d-%H'))-time.mktime(time.strptime('2014-'+i.split(",")[0],'%Y-%m-%d-%H')))/(24*3600)+1, int(i.split(",")[1])) for i in line.strip().split("\t")[1].split(" ")])
        f, inf = [0]*22, 100
        buy = filter(lambda x:x[1]==4, items)
        last = buy[-1][0] if len(buy)!=0 else inf
        a1 = filter(lambda x:x[0]<last and x[1]==1, items) 
        a2 = filter(lambda x:x[0]<last and x[1]==2, items) 
        a3 = filter(lambda x:x[0]<last and x[1]==3, items)
        # 基本统计特征
        f[0] = a2[-1][0] if len(a2)!=0 else inf # 购买后最后一次加入收藏夹距离天数
        f[1] = a3[-1][0] if len(a3)!=0 else inf # 购买后最后一次加入购物车距离天数
        f[2] = len(a1) # 购买后点击次数
        f[3] = len(a2) # 购买后加收次数
        f[4] = len(a3) # 购买后加购次数
        f[5] = len(filter(lambda x:x[0]<=1, items)) # 最后1天交互次数
        f[6] = len(filter(lambda x:x[0]==2, items)) # 倒数第2天交互次数
        f[7] = len(filter(lambda x:x[0]==3, items)) # 倒数第3天交互次数
        f[8] = len(filter(lambda x:x[0]==4, items)) # 倒数第4天交互次数
        f[9] = len(filter(lambda x:x[0]<=7, items)) # 最后7天交互次数
        f[10] = len(buy) # 历史购买次数
        f[11] = last # 最后一次购买距离天数
        f[12] = len(set([item[0] for item in items if item[0]<=3])) # 最后3天内交互天数
        f[13] = len(set([item[0] for item in items if item[0]<=7])) # 最后1周内交互天数
        f[14] = len(set([item[0] for item in items if item[0]<=21])) # 最后3周内交互天数
        f[15] = items[-1][0] if len(items)!=0 else inf # 最后1次交互距离天数
        inter = [len(list(i)) for _,i in itertools.groupby(items, lambda x: x[0])]
        f[16] = len(inter) #交互天数
        f[17] = max(inter) if len(inter)!=0 else 0 # 交互最多的一天交互次数
        f[18] = len(filter(lambda x:x[1]==1, items)) # 总点击次数
        f[19] = len(filter(lambda x:x[1]==2, items)) # 总添加收藏夹次数
        f[20] = len(filter(lambda x:x[1]==3, items)) # 总加购次数
        f[21] = items[0][0]-items[-1][0]+1 if len(items)!=0 else 0 # 第一次交互到最后一次交互的持续时间
        return (uid+"\t"+iid+"\t"+ict+"\t"+"\t".join([str(i) for i in f]))
    else:
        return ("")

In [ ]:
####################################################################################
############################          验证集提取          ############################
####################################################################################

In [ ]:
def extract1_valid(line):
    import time
    (uid, iid, ict), items = line.strip().split("\t")[0].split(" "), filter(lambda x:x[1]==4, [(i.split(",")[0], int(i.split(",")[1])) for i in line.strip().split("\t")[1].split(" ")])
    return (uid+"\t"+iid, ["-".join(item[0].split("-")[:2])+"-0" for item in items])

In [ ]:
subset = {}
for line in fileinput.input(train_set_path):
    subset[line.strip().split(",")[0]] = True
def f(x): return x
counts = lines.filter(lambda x : subset.has_key(x.strip().split("\t")[0].split(" ")[1]))\
              .map(lambda x : extract1_valid(x))\
              .flatMapValues(f)\
              .map(lambda x : x[1]+"\t"+x[0])
output = counts.saveAsTextFile("./data/buy")

In [ ]:
####################################################################################
# LOOP WORK

In [38]:
# target, etime, subset = "12-19-0", "12-18-23", {}
target, etime, subset = "12-18-0", "12-17-23", {}
# target, etime, subset = "12-17-0", "12-16-23", {}
# target, etime, subset = "12-16-0", "12-15-23", {}
# target, etime, subset = "12-15-0", "12-14-23", {}
# target, etime, subset = "12-14-0", "12-13-23", {}
# target, etime, subset = "12-13-0", "12-12-23", {}
# target, etime, subset = "12-12-0", "12-11-23", {}
# target, etime, subset = "12-11-0", "12-10-23", {}
# target, etime, subset = "12-10-0", "12-09-23", {}
# target, etime, subset = "12-09-0", "12-08-23", {}
# target, etime, subset = "12-08-0", "12-07-23", {}
# target, etime, subset = "12-07-0", "12-06-23", {}
# target, etime, subset = "12-06-0", "12-05-23", {}
# target, etime, subset = "12-05-0", "12-04-23", {}
# target, etime, subset = "12-04-0", "12-03-23", {}
# target, etime, subset = "12-03-0", "12-04-23", {}
# target, etime, subset = "12-02-0", "12-01-23", {}
# target, etime, subset = "12-01-0", "11-30-23", {}

In [23]:
for line in fileinput.input(train_set_path):
    subset[line.split(",")[0]] = True
counts = lines.map(lambda x : extract1_item(x))\
              .filter(lambda x : x[0]!="")\
              .groupByKey()\
              .map(lambda x : x[0]+"\t"+extract2_item(x[1]))
output = counts.saveAsTextFile(feature_path+target+"/prod/")

In [33]:
for line in fileinput.input(train_set_path):
        subset[line.split(",")[0]] = True
counts = lines.map(lambda x : extract1_user(x))\
              .groupByKey()\
              .map(lambda x : x[0]+"\t"+extract2_user(x[1]))
output = counts.saveAsTextFile(feature_path+target+"/user/")

In [39]:
for line in fileinput.input(train_set_path):
    subset[line.strip().split(",")[2]] = True
counts = lines.map(lambda x : extract1_user_brand(x))\
              .filter(lambda x : x[0]!="")\
              .groupByKey()\
              .map(lambda x : x[0]+"\t"+extract2_user_brand(x[1]))
output = counts.saveAsTextFile(feature_path+target+"/user_ict/")

In [41]:
for line in fileinput.input(train_set_path):
        subset[line.strip().split(",")[0]] = True
counts = lines.map(lambda x : extract_user_prod(x))\
              .filter(lambda x : x!="")
output = counts.saveAsTextFile(feature_path+target+"/user_prod/")

In [ ]:
####################################################################################
############################           特征合并           ###########################
####################################################################################

In [ ]:
for line in fileinput.input(train_set_path):
    subset[line.strip().split(",")[0]] = True
feature_user_prod = sc.textFile(feature_path+target+'/user_prod', 1).map(lambda x : (x.strip().split("\t")[0]+"\t"+x.strip().split("\t")[2],x.strip()))
feature_user_ict = sc.textFile(feature_path+target+'/user_ict', 1).map(lambda x : (x.strip().split("\t")[0]+"\t"+x.strip().split("\t")[1],"\t".join(x.strip().split("\t")[2:])))
feature_prod = sc.textFile(feature_path+target+'/prod', 1).map(lambda x : (x.strip().split("\t")[0],"\t".join(x.strip().split("\t")[1:])))
feature_user = sc.textFile(feature_path+target+'/user', 1).map(lambda x : (x.strip().split("\t")[0],"\t".join(x.strip().split("\t")[1:])))
counts = feature_user_prod.join(feature_user_ict).map(lambda x:(x[1][0].split("\t")[1],x[1][0]+"\t"+x[1][1]))
counts = counts.join(feature_prod).map(lambda x:(x[1][0].split("\t")[0],x[1][0]+"\t"+x[1][1]))
counts = counts.join(feature_user).map(lambda x:x[1][0]+"\t"+x[1][1])
counts = counts.filter(lambda x : int(x.strip().split("\t")[3:][9])>0 and int(x.strip().split("\t")[3:][17])>=2)
output = counts.saveAsTextFile("./data/prediction/"+target)

In [ ]:
#####################################################################################
#############################             预测            ###########################
#####################################################################################

In [ ]:
# def prediction(line):
#     feature = [float(line.strip().split("\t")[3:][i]) for i in xrange(len(line.strip().split("\t")[3:])) if i in [1,2,4,5,9,12,15,17,18,20,23,27,48,50,62,66]]
#     return (sum([clfs[i].predict_proba(feature)[0][0] for i in xrange(N)]), "\t".join(line.strip().split("\t")[:2]))

# global N
# global clfs

In [ ]:
# N = 3
# import time
# import random
# import pickle
# import fileinput
# time_start = time.time()
# featureset = [line.strip() for line in fileinput.input("./data/feature_list.txt")]
# buyset = {}
# for line in sc.textFile('./data/buy', 1).collect():
#     buyset[line.strip()] = True
# X_train_0, Y_train_0, X_train_1, Y_train_1 = [], [], [], []
# dates = ["12-15-0","12-16-0"]
# for date in dates:
#     for line in sc.textFile('./data/prediction/'+date, 1).collect():
#     # for line in fileinput.input("prediction/"+date+".txt"):
#         if buyset.has_key(date+"\t"+"\t".join(line.strip().split("\t")[:2])):
#             # X_train_1.append([float(i) for i in line.strip().split("\t")[3:]])
#             X_train_1.append([float(line.strip().split("\t")[3:][i]) for i in xrange(len(line.strip().split("\t")[3:])) if i in [1,2,4,5,9,12,15,17,18,20,23,27,48,50,62,66]])
#             Y_train_1.append(1)
#         else:
#             # X_train_0.append([float(i) for i in line.strip().split("\t")[3:]])
#             X_train_0.append([float(line.strip().split("\t")[3:][i]) for i in xrange(len(line.strip().split("\t")[3:])) if i in [1,2,4,5,9,12,15,17,18,20,23,27,48,50,62,66]])
#             Y_train_0.append(0)	
# X_train, Y_train = [[] for i in xrange(N)], [[] for i in xrange(N)]
# for i in xrange(N):
#     for j in random.sample([j for j in xrange(len(X_train_0))], 10*len(X_train_1)):
#         X_train[i].append(X_train_0[j])
#         Y_train[i].append(Y_train_0[j])
#     X_train[i].extend(X_train_1)
#     Y_train[i].extend(Y_train_1)
# print X_train[0][0]
# print Y_train[0][0]
# print len(Y_train[0])
# # clfs = [RandomForestClassifier(n_estimators=20,max_features=10,max_depth=3,random_state=1,class_weight={0:1,1:1}) for i in xrange(N)]
# clfs = [GradientBoostingClassifier(n_estimators=50, max_depth=3, learning_rate=0.1) for i in xrange(N)]
# # criterion = "gini"|"entropy"
# # clf = Pipeline([
# #   ('feature_selection', LinearSVC(penalty="l1", dual=False)),
# #   # ('classification', RandomForestClassifier(n_estimators=20,max_features=30,max_depth=3,random_state=1,class_weight={1:2}))
# #    ('classification', RandomForestClassifier(n_estimators=100,max_features=30,max_depth=3,random_state=1,class_weight={0:5}))
# #   # ('classification', GradientBoostingClassifier(n_estimators=50, max_depth=3, learning_rate=0.1))
# #   # ('classification', SVC(kernel='rbf', probability=True, class_weight={1:10}))
# # ])
# for i in xrange(N):
#     clfs[i].fit(X_train[i],Y_train[i])
# # for i in xrange(len(clf.feature_importances_)):
# #   if clf.feature_importances_[i] >= 0.01:
# #     print i, clf.feature_importances_[i], featureset[i]
# ############################ 备份模型 ############################ 
# # pickle.dump(clf,open('clf.p', 'wb'))
# # clf = pickle.load(open("clf.p","rb"))
# ############################ 备份模型 ############################ 
# print "training finished!~"
# pred = "12-17-0"
# lines = sc.textFile('./data/prediction/'+pred, 1)
# counts = lines.map(lambda x : prediction(x)).sortByKey().collect()
# recall, precision, correct = len([1 for key in buyset.keys() if key.strip().split("\t")[0] == pred]), 400, len([1 for item in counts[:400] if buyset.has_key(pred+"\t"+item[1])])
# print "prediction finished!~"
# print recall, precision, correct, 1.0*correct/recall, 1.0*correct/precision, 2*1.0*correct/recall*1.0*correct/precision/(1.0*correct/recall+1.0*correct/precision)
# with open("tianchi_mobile_recommendation_predict.csv","w") as f:
#     f.write("user_id,item_id\n")
#     for item in counts[:400]:
#         f.write(",".join(item[1].split("\t"))+"\n")
# time_end = time.time()
# print time_end-time_start